### Install mrcnn librarary

In [53]:
#install mrcnn library
!pip install mrcnn

#### Importing Libraries

In [1]:
import sys
import json
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import matplotlib.lines as lines
from matplotlib.patches import Polygon
from mrcnn import utils
from mrcnn import visualize
from mrcnn.visualize import display_images
from mrcnn import model
import mrcnn.model as modellib
from mrcnn.model import log
import cv2
import os
#place custom_1.py inside the folder where this ipynb is run
import custom_1 

Using TensorFlow backend.


### Ground Truth Data Generation

In [4]:
# Load dataset
dataset = custom_1.CustomDataset()
custom_DIR = ROOT_DIRECTORY
#place your json file inside train folder
dataset.load_custom(custom_DIR, "train")
# Must call before using the dataset
dataset.prepare()

print("Image Count: {}".format(len(dataset.image_ids)))
print("Class Count: {}".format(dataset.num_classes))
for i, info in enumerate(dataset.class_info):
    print("{:3}. {:50}".format(i, info['name']))

'\nprint("Image Count: {}".format(len(dataset.image_ids)))\nprint("Class Count: {}".format(dataset.num_classes))\nfor i, info in enumerate(dataset.class_info):\n    print("{:3}. {:50}".format(i, info[\'name\']))\n'

In [5]:
#image_ids = np.random.choice(dataset.image_ids, 60)
os.chdir('ROOT_DIRECTORY\\train_masked')
index = dataset.image_ids
#index=range(1,5)
#print(image_ids)
to_display = []
titles = []
for image_id in index:
    image = dataset.load_image(image_id)
    mask, class_ids = dataset.load_mask(image_id)
    #mask=mask[:,:,-1]
    # Pick top prominent classes in this image
    unique_class_ids = np.unique(class_ids)
    #print(unique_class_ids)
    mask_area = [np.sum(mask[:, :, np.where(class_ids == i)[0]]) for i in unique_class_ids]
    top_ids = [v[0] for v in sorted(zip(unique_class_ids, mask_area),key=lambda r: r[1], reverse=True) if v[1] > 0]
    
    # Generate images and titles
    for i in range(1):
        class_id = top_ids[i] if i < len(top_ids) else -1
        # Pull masks of instances belonging to the same class.
        m = mask[:, :, np.where(class_ids == class_id)[0]]
        m = np.sum(m * np.arange(1, m.shape[-1] +1), -1)
        m[m>1]=1
        #print(np.unique(m))
        to_display.append(m)
        titles.append(dataset.class_names[class_id] if class_id != -1 else "-")
        mask = 'masked_'+dataset.image_info[image_id]['id']
        #print(m.shape)
        plt.imsave(mask,m)

### Read the binary masked image using Opencv
##### Ideally, background will be having the value of 30 and foreground with value of 215.
##### Inorder to make it to 0's and 1's , we will replace 30 with 0 and 215 with 1.

In [10]:
a=cv2.imread('masked_filename.png',0)
print('Befpre Conversion',np.unique(a))
a[a==30]=0
a[a==215]=1
print('After Conversion',np.unique(a))
#plt.imshow(a)

Befpre Conversion [ 30 215]
After Conversion [0 1]
